Problem Statement: PREDICT THE BURNED AREA OF FOREST FIRES WITH NEURAL NETWORKS


In [1]:
import pandas as pd
import numpy as np 
from keras.layers import Dense 
from keras.models import Sequential

In [2]:
forest = pd.read_csv("https://excelr.s3.nl-ams.scw.cloud/DataScience-Assignments/Assignments/SVM/forestfires.csv")

In [3]:
forest.head()

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,daymon,daysat,daysun,daythu,daytue,daywed,monthapr,monthaug,monthdec,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,small
1,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,small
2,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,small
3,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,small
4,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,small


In [4]:
forest.shape

(517, 31)

In [5]:
forest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 31 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   month          517 non-null    object 
 1   day            517 non-null    object 
 2   FFMC           517 non-null    float64
 3   DMC            517 non-null    float64
 4   DC             517 non-null    float64
 5   ISI            517 non-null    float64
 6   temp           517 non-null    float64
 7   RH             517 non-null    int64  
 8   wind           517 non-null    float64
 9   rain           517 non-null    float64
 10  area           517 non-null    float64
 11  dayfri         517 non-null    int64  
 12  daymon         517 non-null    int64  
 13  daysat         517 non-null    int64  
 14  daysun         517 non-null    int64  
 15  daythu         517 non-null    int64  
 16  daytue         517 non-null    int64  
 17  daywed         517 non-null    int64  
 18  monthapr  

In [6]:
from sklearn.preprocessing import LabelEncoder

In [7]:

label_encoder = LabelEncoder()
forest['size_category'] = label_encoder.fit_transform(forest.size_category)

In [8]:
forest = forest.drop(['month','day'], axis = 1)

In [9]:
forest.head()

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,daymon,daysat,daysun,daythu,daytue,daywed,monthapr,monthaug,monthdec,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
1,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1
2,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
3,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
4,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1


In [10]:
forest.describe()

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,daymon,daysat,daysun,daythu,daytue,daywed,monthapr,monthaug,monthdec,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
count,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000
mean,90.644681,110.872340,547.940039,9.021663,18.889168,44.288201,4.017602,0.021663,12.847292,0.164410,0.143133,0.162476,0.183752,0.117988,0.123791,0.104449,0.017408,0.355899,0.017408,0.038685,0.003868,0.061896,0.032882,0.104449,0.003868,0.001934,0.029014,0.332689,0.731141
std,5.520111,64.046482,248.066192,4.559477,5.806625,16.317469,1.791653,0.295959,63.655818,0.371006,0.350548,0.369244,0.387657,0.322907,0.329662,0.306138,0.130913,0.479249,0.130913,0.193029,0.062137,0.241199,0.178500,0.306138,0.062137,0.043980,0.168007,0.471632,0.443796
min,18.700000,1.100000,7.900000,0.000000,2.200000,15.000000,0.400000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,90.200000,68.600000,437.700000,6.500000,15.500000,33.000000,2.700000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,91.600000,108.300000,664.200000,8.400000,19.300000,42.000000,4.000000,0.000000,0.520000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,92.900000,142.400000,713.900000,10.800000,22.800000,53.000000,4.900000,0.000000,6.570000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,96.200000,291.300000,860.600000,56.100000,33.300000,100.000000,9.400000,6.400000,1090.840000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [11]:
forest['size_category'].value_counts()

1    378
0    139
Name: size_category, dtype: int64

In [13]:
x = forest.iloc[:,0:28]
y = forest.iloc[:,28]

In [14]:
x

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,daymon,daysat,daysun,daythu,daytue,daywed,monthapr,monthaug,monthdec,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep
0,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.00,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
2,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.00,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.00,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,6.44,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
513,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,54.29,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
514,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,11.16,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
515,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,0.00,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [15]:
y

0      1
1      1
2      1
3      1
4      1
      ..
512    0
513    0
514    0
515    1
516    1
Name: size_category, Length: 517, dtype: int64

In [16]:
from sklearn.preprocessing import StandardScaler

In [17]:
std = StandardScaler()
x_std = std.fit_transform(x)

#### Hyper tuning of parameters

In [18]:
from sklearn.model_selection import GridSearchCV,KFold
from keras.wrappers.scikit_learn import KerasRegressor      
from tensorflow.keras.optimizers import Adam

#### Creating a framework

In [19]:
def create_model():
    model = Sequential()
    model.add(Dense(15,input_dim = 28,kernel_initializer = 'uniform', activation = 'relu'))
    model.add(Dense(10,kernel_initializer = 'uniform', activation = 'relu'))
    model.add(Dense(1,kernel_initializer = 'uniform', activation = 'sigmoid'))
    
    adam = Adam(lr = 0.01)
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

##### Creating the model

##### Tuning for batch size and epochs

In [20]:
model = KerasRegressor(build_fn = create_model, verbose = 0)

batch_size = [20,40,50]
epochs = [10,60,70]

param_grid = dict(batch_size = batch_size, epochs = epochs)

grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(), verbose = 10)
grid.fit(x_std,y)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] batch_size=20, epochs=10 ........................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


[CV] ........... batch_size=20, epochs=10, score=-0.340, total=   1.9s
[CV] batch_size=20, epochs=10 ........................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.9s remaining:    0.0s


[CV] ........... batch_size=20, epochs=10, score=-0.569, total=   0.9s
[CV] batch_size=20, epochs=10 ........................................


...

[CV] ........... batch_size=20, epochs=60, score=-0.332, total=   3.2s
[CV] batch_size=20, epochs=60 ........................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   16.6s remaining:    0.0s


[CV] ........... batch_size=20, epochs=60, score=-0.509, total=   3.2s
[CV] batch_size=20, epochs=70 ........................................
[CV] ........... batch_size=20, epochs=70, score=-0.020, total=   2.4s
[CV] batch_size=20, epochs=70 ........................................
[CV] ........... batch_size=20, epochs=70, score=-0.352, total=   3.2s
[CV] batch_size=20, epochs=70 ........................................
[CV] ........... batch_size=20, epochs=70, score=-0.181, total=   2.8s
[CV] batch_size=20, epochs=70 ........................................
[CV] ........... batch_size=20, epochs=70, score=-0.255, total=   2.4s
[CV] batch_size=20, epochs=70 ........................................
[CV] ........... batch_size=20, epochs=70, score=-1.340, total=   3.2s
[CV] batch_size=40, epochs=10 ........................................
[CV] ........... batch_size=40, epochs=10, score=-0.475, total=   0.8s
[CV] batch_size=40, epochs=10 ........................................
[CV] .

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:  1.3min finished


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             error_score=nan,
             estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x7f5ee16e68d0>,
             iid='deprecated', n_jobs=None,
             param_grid={'batch_size': [20, 40, 50], 'epochs': [10, 60, 70]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=10)

In [21]:
print('Best: {} with: {}'.format(grid.best_score_ , grid.best_params_))

means = grid.cv_results_['mean_test_score']
std = grid.cv_results_['std_test_score']
params = grid.cv_results_['params']

for mean,stdev,param in zip(means,std,params):
       print('{} {} with : {}'.format(mean,stdev,param))
   

Best: -0.3627262480556965 with: {'batch_size': 20, 'epochs': 60}
-0.583316695690155 0.141595749423946 with : {'batch_size': 20, 'epochs': 10}
-0.3627262480556965 0.1748512580353678 with : {'batch_size': 20, 'epochs': 60}
-0.4295836087316275 0.4677646963989837 with : {'batch_size': 20, 'epochs': 70}
-0.6098673522472382 0.08699083807271447 with : {'batch_size': 40, 'epochs': 10}
-0.5374384000897408 0.25377913902982563 with : {'batch_size': 40, 'epochs': 60}
-0.4742425441741943 0.19507270414176614 with : {'batch_size': 40, 'epochs': 70}
-0.6284257531166076 0.04244511136479865 with : {'batch_size': 50, 'epochs': 10}
-0.5414831459522247 0.21431672036924956 with : {'batch_size': 50, 'epochs': 60}
-0.48090521097183225 0.2071187365458003 with : {'batch_size': 50, 'epochs': 70}


##### Tuning for dropout rate and learning rate

In [22]:
from keras.layers import Dropout

In [24]:

def create_model(dropout_rate,learning_rate):
    model = Sequential()
    model.add(Dense(15,input_dim = 28,kernel_initializer = 'uniform', activation = 'relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(10,kernel_initializer = 'uniform', activation = 'relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1,kernel_initializer = 'uniform', activation = 'sigmoid'))
    
    adam = Adam(lr = 0.01)
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

model = KerasRegressor(build_fn = create_model,batch_size = 30, epochs = 70, verbose = 0)

dropout_rate = [0.1,0.2,0.5]
learning_rate = [0.1,0.01,0.001]

param_grid = dict(dropout_rate = dropout_rate, learning_rate = learning_rate)

grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(), verbose = 10)
grid.fit(x_std,y)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] dropout_rate=0.1, learning_rate=0.1 .............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


[CV]  dropout_rate=0.1, learning_rate=0.1, score=-0.045, total=   2.0s
[CV] dropout_rate=0.1, learning_rate=0.1 .............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.0s remaining:    0.0s


[CV]  dropout_rate=0.1, learning_rate=0.1, score=-0.449, total=   2.0s
[CV] dropout_rate=0.1, learning_rate=0.1 .............................


...

[CV]  dropout_rate=0.1, learning_rate=0.01, score=-0.403, total=   1.9s
[CV] dropout_rate=0.1, learning_rate=0.01 ............................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   22.0s remaining:    0.0s


[CV]  dropout_rate=0.1, learning_rate=0.01, score=-1.025, total=   2.0s
[CV] dropout_rate=0.1, learning_rate=0.001 ...........................
[CV]  dropout_rate=0.1, learning_rate=0.001, score=-0.034, total=   3.3s
[CV] dropout_rate=0.1, learning_rate=0.001 ...........................
[CV]  dropout_rate=0.1, learning_rate=0.001, score=-0.443, total=   3.2s
[CV] dropout_rate=0.1, learning_rate=0.001 ...........................
[CV]  dropout_rate=0.1, learning_rate=0.001, score=-0.406, total=   3.6s
[CV] dropout_rate=0.1, learning_rate=0.001 ...........................
[CV]  dropout_rate=0.1, learning_rate=0.001, score=-0.442, total=   2.0s
[CV] dropout_rate=0.1, learning_rate=0.001 ...........................
[CV]  dropout_rate=0.1, learning_rate=0.001, score=-1.023, total=   1.9s
[CV] dropout_rate=0.2, learning_rate=0.1 .............................
[CV]  dropout_rate=0.2, learning_rate=0.1, score=-0.064, total=   2.0s
[CV] dropout_rate=0.2, learning_rate=0.1 .........................

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:  1.8min finished


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             error_score=nan,
             estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x7f5ed48ce290>,
             iid='deprecated', n_jobs=None,
             param_grid={'dropout_rate': [0.1, 0.2, 0.5],
                         'learning_rate': [0.1, 0.01, 0.001]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=10)

In [25]:
print('Best: {} with: {}'.format(grid.best_score_ , grid.best_params_))

means = grid.cv_results_['mean_test_score']
std = grid.cv_results_['std_test_score']
params = grid.cv_results_['params']

for mean,stdev,param in zip(means,std,params):
       print('{} {} with : {}'.format(mean,stdev,param))

Best: -0.4056734502315521 with: {'dropout_rate': 0.1, 'learning_rate': 0.1}
-0.4056734502315521 0.21294146823982993 with : {'dropout_rate': 0.1, 'learning_rate': 0.1}
-0.5065063804388046 0.311987810210259 with : {'dropout_rate': 0.1, 'learning_rate': 0.01}
-0.4695621855556965 0.3166884191805338 with : {'dropout_rate': 0.1, 'learning_rate': 0.001}
-0.4458386301994324 0.2594376845745775 with : {'dropout_rate': 0.2, 'learning_rate': 0.1}
-0.4734279975295067 0.26444269403764215 with : {'dropout_rate': 0.2, 'learning_rate': 0.01}
-0.4708801358938217 0.31517072642665905 with : {'dropout_rate': 0.2, 'learning_rate': 0.001}
-0.5008348494768142 0.22422097911537545 with : {'dropout_rate': 0.5, 'learning_rate': 0.1}
-0.47250410467386245 0.22454750535520718 with : {'dropout_rate': 0.5, 'learning_rate': 0.01}
-0.44525673538446425 0.1973257080446334 with : {'dropout_rate': 0.5, 'learning_rate': 0.001}


##### Activation Function and kernel initializer

In [ ]:
def create_model(activation_function,init):
    model = Sequential()
    model.add(Dense(15,input_dim = 28,kernel_initializer = init, activation = activation_function))
    model.add(Dropout(0.5))
    model.add(Dense(10,kernel_initializer = init, activation = activation_function))
    model.add(Dropout(0.5))
    model.add(Dense(1,kernel_initializer = 'uniform', activation = 'sigmoid'))
    
    adam = Adam(lr = 0.0001)
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

model = KerasRegressor(build_fn = create_model,batch_size = 30, epochs = 70, verbose = 0)

activation_function = ['relu','maxout','elu','tanh','sigmoid']
init = ['uniform','normal','zero']

param_grid = dict(activation_function = activation_function, init = init)

grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(), verbose = 10)
grid.fit(x_std,y)

Fitting 5 folds for each of 15 candidates, totalling 75 fits
[CV] activation_function=relu, init=uniform ..........................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


[CV]  activation_function=relu, init=uniform, score=-0.108, total=   2.0s
[CV] activation_function=relu, init=uniform ..........................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.0s remaining:    0.0s


[CV]  activation_function=relu, init=uniform, score=-0.451, total=   3.3s
[CV] activation_function=relu, init=uniform ..........................


...

[CV]  activation_function=relu, init=normal, score=-0.527, total=   2.0s
[CV] activation_function=relu, init=normal ...........................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   24.5s remaining:    0.0s


[CV]  activation_function=relu, init=normal, score=-0.703, total=   3.2s
[CV] activation_function=relu, init=zero .............................
[CV]  activation_function=relu, init=zero, score=-0.458, total=   2.0s
[CV] activation_function=relu, init=zero .............................
[CV]  activation_function=relu, init=zero, score=-0.581, total=   2.4s
[CV] activation_function=relu, init=zero .............................
[CV]  activation_function=relu, init=zero, score=-0.743, total=   3.2s
[CV] activation_function=relu, init=zero .............................
[CV]  activation_function=relu, init=zero, score=-0.627, total=   2.0s
[CV] activation_function=relu, init=zero .............................
[CV]  activation_function=relu, init=zero, score=-0.614, total=   2.1s
[CV] activation_function=maxout, init=uniform ........................
[CV]  activation_function=maxout, init=uniform, score=nan, total=   0.0s
[CV] activation_function=maxout, init=uniform ........................
[C

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Unknown activation function: maxout. Please ensure this object is passed to the `custom_objects` argument. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.

  FitFailedWarning)


[CV]  activation_function=elu, init=uniform, score=-0.038, total=   3.2s
[CV] activation_function=elu, init=uniform ...........................
[CV]  activation_function=elu, init=uniform, score=-0.342, total=   2.0s
[CV] activation_function=elu, init=uniform ...........................
[CV]  activation_function=elu, init=uniform, score=-0.718, total=   2.0s
[CV] activation_function=elu, init=uniform ...........................
[CV]  activation_function=elu, init=uniform, score=-0.224, total=   3.3s
[CV] activation_function=elu, init=uniform ...........................
[CV]  activation_function=elu, init=uniform, score=-0.522, total=   2.3s
[CV] activation_function=elu, init=normal ............................
[CV]  activation_function=elu, init=normal, score=-0.042, total=   2.0s
[CV] activation_function=elu, init=normal ............................
[CV]  activation_function=elu, init=normal, score=-0.390, total=   3.2s
[CV] activation_function=elu, init=normal .......................

In [27]:
print('Best: {} with: {}'.format(grid.best_score_ , grid.best_params_))

means = grid.cv_results_['mean_test_score']
std = grid.cv_results_['std_test_score']
params = grid.cv_results_['params']

for mean,stdev,param in zip(means,std,params):
       print('{} {} with : {}'.format(mean,stdev,param))

Best: -0.29875795766711233 with: {'activation_function': 'tanh', 'init': 'normal'}
-0.4657146975398064 0.21999307942460689 with : {'activation_function': 'relu', 'init': 'uniform'}
-0.5058907210826874 0.1976358683861132 with : {'activation_function': 'relu', 'init': 'normal'}
-0.6045683205127717 0.09150088093784811 with : {'activation_function': 'relu', 'init': 'zero'}
nan nan with : {'activation_function': 'maxout', 'init': 'uniform'}
nan nan with : {'activation_function': 'maxout', 'init': 'normal'}
nan nan with : {'activation_function': 'maxout', 'init': 'zero'}
-0.36881601363420485 0.23498677159721337 with : {'activation_function': 'elu', 'init': 'uniform'}
-0.3773676559329033 0.21006956959403209 with : {'activation_function': 'elu', 'init': 'normal'}
-0.6153757750988007 0.1352830260304457 with : {'activation_function': 'elu', 'init': 'zero'}
-0.30092202462255957 0.17018345146423772 with : {'activation_function': 'tanh', 'init': 'uniform'}
-0.29875795766711233 0.15735973266179396 w

##### optimizing number of neurons

In [28]:
def create_model(neuron1,neuron2):
    model = Sequential()
    model.add(Dense(neuron1,input_dim = 28,kernel_initializer = 'uniform', activation = 'tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(neuron2,kernel_initializer = 'uniform', activation = 'tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1,kernel_initializer = 'uniform', activation = 'sigmoid'))
    
    adam = Adam(lr = 0.0001)
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

model = KerasRegressor(build_fn = create_model,batch_size = 30, epochs = 70, verbose = 0)

neuron1 = [10,12,15,20]
neuron2 = [5,8,10,14]

param_grid = dict(neuron1 = neuron1, neuron2 = neuron2)

grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(), verbose = 10)
grid.fit(x_std,y)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] neuron1=10, neuron2=5 ...........................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


[CV] .............. neuron1=10, neuron2=5, score=-0.107, total=   1.8s
[CV] neuron1=10, neuron2=5 ...........................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.8s remaining:    0.0s


[CV] .............. neuron1=10, neuron2=5, score=-0.397, total=   2.0s
[CV] neuron1=10, neuron2=5 ...........................................


...

[CV] .............. neuron1=10, neuron2=8, score=-0.276, total=   2.0s
[CV] neuron1=10, neuron2=8 ...........................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   17.5s remaining:    0.0s


[CV] .............. neuron1=10, neuron2=8, score=-0.436, total=   1.8s
[CV] neuron1=10, neuron2=10 ..........................................
[CV] ............. neuron1=10, neuron2=10, score=-0.039, total=   2.0s
[CV] neuron1=10, neuron2=10 ..........................................
[CV] ............. neuron1=10, neuron2=10, score=-0.328, total=   2.0s
[CV] neuron1=10, neuron2=10 ..........................................
[CV] ............. neuron1=10, neuron2=10, score=-0.553, total=   1.9s
[CV] neuron1=10, neuron2=10 ..........................................
[CV] ............. neuron1=10, neuron2=10, score=-0.268, total=   1.9s
[CV] neuron1=10, neuron2=10 ..........................................
[CV] ............. neuron1=10, neuron2=10, score=-0.481, total=   2.0s
[CV] neuron1=10, neuron2=14 ..........................................
[CV] ............. neuron1=10, neuron2=14, score=-0.031, total=   1.9s
[CV] neuron1=10, neuron2=14 ..........................................
[CV] .

[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:  3.1min finished


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             error_score=nan,
             estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x7f5ed483aad0>,
             iid='deprecated', n_jobs=None,
             param_grid={'neuron1': [10, 12, 15, 20],
                         'neuron2': [5, 8, 10, 14]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=10)

In [29]:
print('Best: {} with: {}'.format(grid.best_score_ , grid.best_params_))

means = grid.cv_results_['mean_test_score']
std = grid.cv_results_['std_test_score']
params = grid.cv_results_['params']

for mean,stdev,param in zip(means,std,params):
       print('{} {} with : {}'.format(mean,stdev,param))

Best: -0.2819236271083355 with: {'neuron1': 20, 'neuron2': 10}
-0.3384720519185066 0.12042211402196114 with : {'neuron1': 10, 'neuron2': 5}
-0.3161181673407555 0.148407395001948 with : {'neuron1': 10, 'neuron2': 8}
-0.33393165320158 0.17942070016867373 with : {'neuron1': 10, 'neuron2': 10}
-0.31971227638423444 0.21050236007436607 with : {'neuron1': 10, 'neuron2': 14}
-0.32389363646507263 0.13354778083801278 with : {'neuron1': 12, 'neuron2': 5}
-0.32690157517790797 0.1605704278325416 with : {'neuron1': 12, 'neuron2': 8}
-0.3104529194533825 0.16902392719029624 with : {'neuron1': 12, 'neuron2': 10}
-0.30658262185752394 0.20567121344134376 with : {'neuron1': 12, 'neuron2': 14}
-0.31013218611478804 0.12649605772235567 with : {'neuron1': 15, 'neuron2': 5}
-0.32391020357608796 0.16543730468902787 with : {'neuron1': 15, 'neuron2': 8}
-0.30943878218531606 0.17653053174064348 with : {'neuron1': 15, 'neuron2': 10}
-0.29934842213988305 0.2025919476207915 with : {'neuron1': 15, 'neuron2': 14}
-0.29

##### Building the final model 

In [30]:
from sklearn.model_selection import cross_val_score

In [ ]:
def create_model():
    model = Sequential()
    model.add(Dense(20,input_dim = 28,kernel_initializer = 'uniform', activation = 'tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(14,kernel_initializer = 'uniform', activation = 'tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1,kernel_initializer = 'uniform', activation = 'sigmoid'))
    
    adam = Adam(lr = 0.0001)
    model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics = ['accuracy'])
    return model

model = KerasRegressor(build_fn = create_model,batch_size = 30, epochs = 70, verbose = 10)

model.fit(x_std,y)

kfold = KFold(n_splits=30)
results = cross_val_score(model, x_std, y, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))


Epoch 1/70
Epoch 2/70
Epoch 3/70
Epoch 4/70
Epoch 5/70
Epoch 6/70
Epoch 7/70
Epoch 8/70
Epoch 9/70
Epoch 10/70
Epoch 11/70
Epoch 12/70
Epoch 13/70
Epoch 14/70
Epoch 15/70
Epoch 16/70
Epoch 17/70
Epoch 18/70
Epoch 19/70
Epoch 20/70
Epoch 21/70
Epoch 22/70
Epoch 23/70
Epoch 24/70
Epoch 25/70
Epoch 26/70
Epoch 27/70
Epoch 28/70
Epoch 29/70
Epoch 30/70
Epoch 31/70
Epoch 32/70
Epoch 33/70
Epoch 34/70
Epoch 35/70
Epoch 36/70
Epoch 37/70
Epoch 38/70
Epoch 39/70
Epoch 40/70
Epoch 41/70
Epoch 42/70
Epoch 43/70
Epoch 44/70
Epoch 45/70
Epoch 46/70
Epoch 47/70
Epoch 48/70
Epoch 49/70
Epoch 50/70
Epoch 51/70
Epoch 52/70
Epoch 53/70
Epoch 54/70
Epoch 55/70
Epoch 56/70
Epoch 57/70
Epoch 58/70
Epoch 59/70
Epoch 60/70
Epoch 61/70
Epoch 62/70
Epoch 63/70
Epoch 64/70
Epoch 65/70
Epoch 66/70
Epoch 67/70
Epoch 68/70
Epoch 69/70
Epoch 70/70
Epoch 1/70
Epoch 2/70
Epoch 3/70
Epoch 4/70
Epoch 5/70
Epoch 6/70
Epoch 7/70
Epoch 8/70
Epoch 9/70
Epoch 10/70
Epoch 11/70
Epoch 12/70
Epoch 13/70
Epoch 14/70
Epoch 15/7

Epoch 65/70
Epoch 66/70
Epoch 67/70
Epoch 68/70
Epoch 69/70
Epoch 70/70
Epoch 1/70
Epoch 2/70
Epoch 3/70
Epoch 4/70
Epoch 5/70
Epoch 6/70
Epoch 7/70
Epoch 8/70
Epoch 9/70
Epoch 10/70
Epoch 11/70
Epoch 12/70
Epoch 13/70
Epoch 14/70
Epoch 15/70
Epoch 16/70
Epoch 17/70
Epoch 18/70
Epoch 19/70
Epoch 20/70
Epoch 21/70
Epoch 22/70
Epoch 23/70
Epoch 24/70
Epoch 25/70
Epoch 26/70
Epoch 27/70
Epoch 28/70
Epoch 29/70
Epoch 30/70
Epoch 31/70
Epoch 32/70
Epoch 33/70
Epoch 34/70
Epoch 35/70
Epoch 36/70
Epoch 37/70
Epoch 38/70
Epoch 39/70
Epoch 40/70
Epoch 41/70
Epoch 42/70
Epoch 43/70
Epoch 44/70
Epoch 45/70
Epoch 46/70
Epoch 47/70
Epoch 48/70
Epoch 49/70
Epoch 50/70
Epoch 51/70
Epoch 52/70
Epoch 53/70
Epoch 54/70
Epoch 55/70
Epoch 56/70
Epoch 57/70
Epoch 58/70
Epoch 59/70
Epoch 60/70
Epoch 61/70
Epoch 62/70
Epoch 63/70
Epoch 64/70
Epoch 65/70
Epoch 66/70
Epoch 67/70
Epoch 68/70
Epoch 69/70
Epoch 70/70
Epoch 1/70
Epoch 2/70
Epoch 3/70
Epoch 4/70
Epoch 5/70
Epoch 6/70
Epoch 7/70
Epoch 8/70
Epoch 9/7

Epoch 62/70
Epoch 63/70
Epoch 64/70
Epoch 65/70
Epoch 66/70
Epoch 67/70
Epoch 68/70
Epoch 69/70
Epoch 70/70
Epoch 1/70
Epoch 2/70
Epoch 3/70
Epoch 4/70
Epoch 5/70
Epoch 6/70
Epoch 7/70
Epoch 8/70
Epoch 9/70
Epoch 10/70
Epoch 11/70
Epoch 12/70
Epoch 13/70
Epoch 14/70
Epoch 15/70
Epoch 16/70
Epoch 17/70
Epoch 18/70
Epoch 19/70
Epoch 20/70
Epoch 21/70
Epoch 22/70
Epoch 23/70
Epoch 24/70
Epoch 25/70
Epoch 26/70
Epoch 27/70
Epoch 28/70
Epoch 29/70
Epoch 30/70
Epoch 31/70
Epoch 32/70
Epoch 33/70
Epoch 34/70
Epoch 35/70
Epoch 36/70
Epoch 37/70
Epoch 38/70
Epoch 39/70
Epoch 40/70
Epoch 41/70
Epoch 42/70
Epoch 43/70
Epoch 44/70
Epoch 45/70
Epoch 46/70
Epoch 47/70
Epoch 48/70
Epoch 49/70
Epoch 50/70
Epoch 51/70
Epoch 52/70
Epoch 53/70
Epoch 54/70
Epoch 55/70
Epoch 56/70
Epoch 57/70
Epoch 58/70
Epoch 59/70
Epoch 60/70
Epoch 61/70
Epoch 62/70
Epoch 63/70
Epoch 64/70
Epoch 65/70
Epoch 66/70
Epoch 67/70
Epoch 68/70
Epoch 69/70
Epoch 70/70
Epoch 1/70
Epoch 2/70
Epoch 3/70
Epoch 4/70
Epoch 5/70
Epoch 

Epoch 57/70
Epoch 58/70
Epoch 59/70
Epoch 60/70
Epoch 61/70
Epoch 62/70
Epoch 63/70
Epoch 64/70
Epoch 65/70
Epoch 66/70
Epoch 67/70
Epoch 68/70
Epoch 69/70
Epoch 70/70
Epoch 1/70
Epoch 2/70
Epoch 3/70
Epoch 4/70
Epoch 5/70
Epoch 6/70
Epoch 7/70
Epoch 8/70
Epoch 9/70
Epoch 10/70
Epoch 11/70
Epoch 12/70
Epoch 13/70
Epoch 14/70
Epoch 15/70
Epoch 16/70
Epoch 17/70
Epoch 18/70
Epoch 19/70
Epoch 20/70
Epoch 21/70
Epoch 22/70
Epoch 23/70
Epoch 24/70
Epoch 25/70
Epoch 26/70
Epoch 27/70
Epoch 28/70
Epoch 29/70
Epoch 30/70
Epoch 31/70
Epoch 32/70
Epoch 33/70
Epoch 34/70
Epoch 35/70
Epoch 36/70
Epoch 37/70
Epoch 38/70
Epoch 39/70
Epoch 40/70
Epoch 41/70
Epoch 42/70
Epoch 43/70
Epoch 44/70
Epoch 45/70
Epoch 46/70
Epoch 47/70
Epoch 48/70
Epoch 49/70
Epoch 50/70
Epoch 51/70
Epoch 52/70
Epoch 53/70
Epoch 54/70
Epoch 55/70
Epoch 56/70
Epoch 57/70
Epoch 58/70
Epoch 59/70
Epoch 60/70
Epoch 61/70
Epoch 62/70
Epoch 63/70
Epoch 64/70
Epoch 65/70
Epoch 66/70
Epoch 67/70
Epoch 68/70
Epoch 69/70
Epoch 70/70
R

Mean squared error is 0.08